In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv('data/spam.csv', encoding='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
del df['Unnamed: 2']
del df['Unnamed: 3']
del df['Unnamed: 4']
df['v1'] = df['v1'].replace(['ham','spam'],[0,1])
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
# 데이터 중복 제거
df = df.drop_duplicates('v2', keep='first')

In [8]:
df.groupby('v1').size()

v1
0    4516
1     653
dtype: int64

In [9]:
X_data = df['v2']
y_data = df['v1']
print('메일 본문의 개수: {}'.format(len(X_data)))
print('레이블의 개수: {}'.format(len(y_data)))

메일 본문의 개수: 5169
레이블의 개수: 5169


## Naive Bayes 분류기 적용

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB  # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score     # 정확도 계산

In [12]:
# 테스트용 데이터 20% 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X_data, y_data, test_size=.2, random_state=2020)

In [34]:
# 훈련 데이터를 BoW로 만들어줌
spamvector = CountVectorizer()
X_train_dtm = spamvector.fit_transform(X_train)
print(X_train_dtm.shape)

(4135, 7658)


In [35]:
model = MultinomialNB()
model.fit(X_train_dtm, y_train)
# alpha=1.0은 라플라스 스무딩이 적용되었음을 의미

MultinomialNB()

In [36]:
X_test_dtm = spamvector.transform(X_test) # 테스트 데이터를 DTM으로 변환
print(X_test_dtm.shape)

(1034, 7658)


In [37]:
predicted = model.predict(X_test_dtm) # 테스트 데이터에 대한 예측
print("정확도: %.4f" % accuracy_score(y_test, predicted)) # 예측값과 실제값 비교

정확도: 0.9758


In [38]:
import joblib
joblib.dump(model, 'model/spam_nb.pkl')

['model/spam_nb.pkl']

In [39]:
joblib.dump(spamvector, 'model/spam_nb_dtm.pkl')

['model/spam_nb_dtm.pkl']

In [40]:
review = '''Oh k...i'm watching here:)
'''
review_vect = spamvector.transform([review])
model.predict(review_vect)[0]

0